In [1]:
import numpy as np
import scipy.special as scp
from collections import deque
import random

In [2]:
class Task:
    
    def __init__(self, T0 = 1, q = 3 / 5, Ybar = 10, N = 20, alpha = 0.5, rho  = 0.8):
        self.T = 0
        if(np.random.uniform() < q):
            self.T = T0
        else:
            self.T = round(T0 - Ybar * np.log(np.random.uniform()))
        beta = (N * rho * (T0 + (1 - q) * Ybar)) / 2
        EX = beta * scp.gamma(1 + 1 / alpha)
        aux = round(beta * ((-np.log(np.random.uniform())) ** (1 / alpha)))
        self.X = max([1, min([100 * EX, aux])])
        self.start_time = 0
        self.end_time = 0
        
    def copy(self):
        to_return = Task()
        to_return.T = self.T
        to_return.X = self.X
        to_return.start_time = self.start_time
        to_return.end_time = self.end_time
        return(to_return)
        
class Server:
    
    def __init__(self):
        self.queue = deque()
        self.current_task = None
        
    def task_remaining_time(self):
        if(self.current_task is None):
            return(float("+Inf"))
        else:
            return(self.current_task.X)
        
    def add_task(self, task):
        if(self.current_task is None):
            self.current_task = task
        else:
            self.queue.append(task)
        
    def go_next_task(self):
        previous_task = self.current_task
        if(len(self.queue) == 0):
            self.current_task = None
        else:
            self.current_task = self.queue.pop()
        return(previous_task)
    
    def length(self):
        if(self.current_task is None):
            return(0)
        else:
            return(len(self.queue) + 1)
    
    def all_remaining_time(self):
        if(self.current_task is None):
            return(0)
        to_return = self.current_task.X
        to_return = to_return + sum([task.X for task in list(self.queue)])
        return(to_return)
    
    def mean_remaining_time(self):
        if(self.current_task is None):
            return(0)
        else:
            return(self.all_remaining_time() / self.length())
    
    def max_rem_time(self):
        if(self.current_task is None):
            return(0)
        maximus = self.current_task.X
        for task in list(self.queue):
            if(task.X > maximus):
                maximus = task.X
        return(maximus)

In [3]:
def JSQ_simulation(messages, T0 = 1, q = 3 / 5, Ybar = 10, N = 20, alpha = 0.5, rho = 0.8):
    servers = [Server() for _ in range(N)]
    if(isinstance(messages, int)):
        tasks = deque([Task(T0, 1, Ybar, N, alpha, rho) for _ in range(messages)])
    else:
        tasks = deque(messages)
    times = []
    next_task = tasks.pop()
    current_time = 0
    while(not next_task is None):
        # print("Next task : " + str(next_task.T) + " Service time : " + str(next_task.X))
        # for index in range(len(servers)):
        #   to_print = ""
        #   if(not servers[index].current_task is None):
        #       to_print = to_print + " " + str(servers[index].current_task.X)
        #   for element in list(servers[index].queue):
        #       to_print = to_print + " | " + str(element.X)
        #   print("Server - " + str(index) + to_print)
        first_server_stopped = min(servers, key = lambda x : x.task_remaining_time()).task_remaining_time()
        servers_stopped = [server for server in servers if server.task_remaining_time() == first_server_stopped]
        time = min([first_server_stopped, next_task.T])
        current_time = current_time + time
        changed_task = False
        for server in servers:
            if(server.task_remaining_time() != first_server_stopped and not server.current_task is None):
                server.current_task.X = server.current_task.X - time
            elif(server.task_remaining_time() == first_server_stopped and time != first_server_stopped and first_server_stopped != float("+Inf")):
                server.current_task.X = server.current_task.X - time
        if(first_server_stopped <= next_task.T):
            for server in servers_stopped:
                completed_task = server.go_next_task()
                times.append(current_time - completed_task.start_time)
        if(next_task.T <= first_server_stopped):
            chosen_server = min(servers, key = lambda x : x.length())
            chosen_server.add_task(next_task)
            next_task.start_time = current_time
            if(len(tasks) == 0):
                next_task = None
            else:
                next_task = tasks.pop()
            changed_task = True
        if(not changed_task):
            next_task.T = next_task.T - time
    for server in servers:
        other_current_time = current_time
        if(not server.current_task is None):
            times.append(current_time + server.current_task.X - server.current_task.start_time)
            other_current_time = current_time + server.current_task.X
        for task in list(server.queue):
            other_current_time = other_current_time + task.X
            times.append(other_current_time - task.start_time)
    return(times)

In [4]:
def JBT_simulation(messages, d = 3, T = None, T0 = 1, q = 3 / 5, Ybar = 10, N = 20, alpha = 0.5, rho = 0.8):
    if(T is None):
        T = 1000 * T0
    threshold = 1
    servers = [Server() for _ in range(N)]
    if(isinstance(messages, int)):
        tasks = deque([Task(T0, 1, Ybar, N, alpha, rho) for _ in range(messages)])
    else:
        tasks = deque(messages)
    times = []
    next_task = tasks.pop()
    current_time = 0
    while(not next_task is None):
        first_server_stopped = min(servers, key = lambda x : x.task_remaining_time()).task_remaining_time()
        servers_stopped = [server for server in servers if server.task_remaining_time() == first_server_stopped]
        time = min([first_server_stopped, next_task.T])
        if(current_time % T != 0 and (current_time + time) % T == 0):
            random_servers = random.choices(servers, k = d)
            threshold = min([serv.length() for serv in random_servers])
        current_time = current_time + time
        changed_task = False
        for server in servers:
            if(server.task_remaining_time() != first_server_stopped and not server.current_task is None):
                server.current_task.X = server.current_task.X - time
            elif(server.task_remaining_time() == first_server_stopped and time != first_server_stopped and first_server_stopped != float("+Inf")):
                server.current_task.X = server.current_task.X - time
        if(first_server_stopped <= next_task.T):
            for server in servers_stopped:
                completed_task = server.go_next_task()
                times.append(current_time - completed_task.start_time)
        if(next_task.T <= first_server_stopped):
            below_threshold = [server for server in servers if len(server.queue) <= 2]
            if(len(below_threshold) == 0):
                chosen_server = random.choice(servers)
            else:
                chosen_server = random.choice(below_threshold)
            chosen_server.add_task(next_task)
            next_task.start_time = current_time
            if(len(tasks) == 0):
                next_task = None
            else:
                next_task = tasks.pop()
            changed_task = True
        if(not changed_task):
            next_task.T = next_task.T - time
    for server in servers:
        other_current_time = current_time
        if(not server.current_task is None):
            times.append(current_time + server.current_task.X - server.current_task.start_time)
            other_current_time = current_time + server.current_task.X
        for task in list(server.queue):
            other_current_time = other_current_time + task.X
            times.append(other_current_time - task.start_time)
    return(times)

In [5]:
def POD_simulation(messages, d = 3, T0 = 1, q = 3 / 5, Ybar = 10, N = 20, alpha = 0.5, rho = 0.8):
    servers = [Server() for _ in range(N)]
    if(isinstance(messages, int)):
        tasks = deque([Task(T0, 1, Ybar, N, alpha, rho) for _ in range(messages)])
    else:
        tasks = deque(messages)
    times = []
    next_task = tasks.pop()
    current_time = 0
    while(not next_task is None):
        first_server_stopped = min(servers, key = lambda x : x.task_remaining_time()).task_remaining_time()
        servers_stopped = [server for server in servers if server.task_remaining_time() == first_server_stopped]
        time = min([first_server_stopped, next_task.T])
        current_time = current_time + time
        changed_task = False
        for server in servers:
            if(server.task_remaining_time() != first_server_stopped and not server.current_task is None):
                server.current_task.X = server.current_task.X - time
            elif(server.task_remaining_time() == first_server_stopped and time != first_server_stopped and first_server_stopped != float("+Inf")):
                server.current_task.X = server.current_task.X - time
        if(first_server_stopped <= next_task.T):
            for server in servers_stopped:
                completed_task = server.go_next_task()
                times.append(current_time - completed_task.start_time)
        if(next_task.T <= first_server_stopped):
            random_servers = random.choices(servers, k = d)
            chosen_server = min(random_servers, key = lambda x : len(x.queue))
            chosen_server.add_task(next_task)
            next_task.start_time = current_time
            if(len(tasks) == 0):
                next_task = None
            else:
                next_task = tasks.pop()
            changed_task = True
        if(not changed_task):
            next_task.T = next_task.T - time
    for server in servers:
        other_current_time = current_time
        if(not server.current_task is None):
            times.append(current_time + server.current_task.X - server.current_task.start_time)
            other_current_time = current_time + server.current_task.X
        for task in list(server.queue):
            other_current_time = other_current_time + task.X
            times.append(other_current_time - task.start_time)
    return(times)

In [6]:
def NINA_simulation(messages, m = None, threshold_quantile = 0.5, T0 = 1, q = 3 / 5, Ybar = 10, N = 20, alpha = 0.5, rho = 8):
    if(m is None):
        m = N // 2
    light_servers = [Server() for _ in range(m)]
    heavy_servers = [Server() for _ in range(N - m)]
    servers = [server for server in light_servers]
    servers.extend(heavy_servers)
    if(isinstance(messages, int)):
        tasks = deque([Task(T0, 1, Ybar, N, alpha, rho) for _ in range(messages)])
    else:
        tasks = deque(messages)
    times = []
    next_task = tasks.pop()
    observed_times = [next_task.X]
    current_time = 0
    while(not next_task is None):
        first_server_stopped = min(servers, key = lambda x : x.task_remaining_time()).task_remaining_time()
        servers_stopped = [server for server in servers if server.task_remaining_time() == first_server_stopped]
        time = min([first_server_stopped, next_task.T])
        current_time = current_time + time
        changed_task = False
        for server in servers:
            if(server.task_remaining_time() != first_server_stopped and not server.current_task is None):
                server.current_task.X = server.current_task.X - time
            elif(server.task_remaining_time() == first_server_stopped and time != first_server_stopped and first_server_stopped != float("+Inf")):
                server.current_task.X = server.current_task.X - time
        if(first_server_stopped <= next_task.T):
            for server in servers_stopped:
                completed_task = server.go_next_task()
                times.append(current_time - completed_task.start_time)
        if(next_task.T <= first_server_stopped):
            threshold = np.quantile(observed_times[-1000:], threshold_quantile)
            if(next_task.X > threshold):
                chosen_server = min(heavy_servers, key = lambda x : len(x.queue))
            else:
                chosen_server = min(light_servers, key = lambda x : len(x.queue))
            chosen_server.add_task(next_task)
            next_task.start_time = current_time
            if(len(tasks) == 0):
                next_task = None
            else:
                next_task = tasks.pop()
                observed_times.append(next_task.X)
            changed_task = True
        if(not changed_task):
            next_task.T = next_task.T - time
    for server in servers:
        other_current_time = current_time
        if(not server.current_task is None):
            times.append(current_time + server.current_task.X - server.current_task.start_time)
            other_current_time = current_time + server.current_task.X
        for task in list(server.queue):
            other_current_time = other_current_time + task.X
            times.append(other_current_time - task.start_time)
    return(times)

In [7]:
def ALT_simulation(messages, minimize = lambda server : server.all_remaining_time(), T0 = 1, q = 3 / 5, Ybar = 10, N = 20, alpha = 0.5, rho = 0.8):
    servers = [Server() for _ in range(N)]
    if(isinstance(messages, int)):
        tasks = deque([Task(T0, 1, Ybar, N, alpha, rho) for _ in range(messages)])
    else:
        tasks = deque(messages)
    times = []
    next_task = tasks.pop()
    current_time = 0
    while(not next_task is None):
        #print("Next task : " + str(next_task.T) + " Service time : " + str(next_task.X))
        #for index in range(len(servers)):
        #   to_print = ""
        #   if(not servers[index].current_task is None):
        #       to_print = to_print + " " + str(servers[index].current_task.X)
        #   for element in list(servers[index].queue):
        #       to_print = to_print + " | " + str(element.X)
        #   print("Server - " + str(index) + to_print)
        first_server_stopped = min(servers, key = lambda x : x.task_remaining_time()).task_remaining_time()
        servers_stopped = [server for server in servers if server.task_remaining_time() == first_server_stopped]
        time = min([first_server_stopped, next_task.T])
        current_time = current_time + time
        changed_task = False
        for server in servers:
            if(server.task_remaining_time() != first_server_stopped and not server.current_task is None):
                server.current_task.X = server.current_task.X - time
            elif(server.task_remaining_time() == first_server_stopped and time != first_server_stopped and first_server_stopped != float("+Inf")):
                server.current_task.X = server.current_task.X - time
        if(first_server_stopped <= next_task.T):
            for server in servers_stopped:
                completed_task = server.go_next_task()
                times.append(current_time - completed_task.start_time)
        if(next_task.T <= first_server_stopped):
            min_length = min([serv.length() for serv in servers])
            to_pick = [serv for serv in servers if serv.length() == min_length]
            chosen_server = min(to_pick, key = lambda x : minimize(x))
            chosen_server.add_task(next_task)
            next_task.start_time = current_time
            if(len(tasks) == 0):
                next_task = None
            else:
                next_task = tasks.pop()
            changed_task = True
        if(not changed_task):
            next_task.T = next_task.T - time
    for server in servers:
        other_current_time = current_time
        if(not server.current_task is None):
            times.append(current_time + server.current_task.X - server.current_task.start_time)
            other_current_time = current_time + server.current_task.X
        for task in list(server.queue):
            other_current_time = other_current_time + task.X
            times.append(other_current_time - task.start_time)
    return(times)

In [400]:
messages = [Task() for _ in range(100000)]

In [ ]:
FAST = ALT_simulation([task.copy() for task in messages])

In [ ]:
FASTER = ALT_simulation([task.copy() for task in messages], minimize = lambda x : x.max_rem_time())

In [ ]:
JSQ = JSQ_simulation([task.copy() for task in messages])

In [ ]:
np.mean(FAST)

In [ ]:
np.mean(FASTER)

In [ ]:
np.mean(JSQ)

In [ ]:
#NINA
rho = []
times1 = []
sd1 = []
for ro in list(np.linspace(0.8,0.99,70)):
    temp1 = []
    for i in range(15):
        temp1.append(np.mean(JSQ_simulation(1000000, T0 = 1, q = 3 / 5, Ybar = 10, N = 20, alpha = 0.5, rho = ro)))
    times1.append(np.mean(temp1))
    sd1.append(np.std(temp1)/np.sqrt(15))
    #WE NEED THE MEAN OVERHAD ALSOOOO
    rho.append(ro)    
d1 = {'rho':rho, 'meanDelay':times1, 'sd':sd1}
d1 = pd.DataFrame.from_dict(d1)
d4.to_csv('d1.csv', index = False)

In [ ]:
#DAVIDE
rho = []
times2 = []
sd2 = []
for ro in list(np.linspace(0.8,0.99,70)):
    temp2 = []
    for i in range(15):
        temp2.append(np.mean(JBT_simulation(1000000, T0 = 1, q = 3 / 5, Ybar = 10, N = 20, alpha = 0.5, rho = ro)))
    times2.append(np.mean(temp2))
    sd2.append(np.std(temp2)/np.sqrt(15))
    #WE NEED THE MEAN OVERHAD ALSOOOO
    rho.append(ro)    
d2 = {'rho':rho, 'meanDelay':times2, 'sd':sd2}
d2 = pd.DataFrame.from_dict(d2)
d2.to_csv('d2.csv', index = False)

In [ ]:
#LEO
rho = []
times3 = []
sd3 = []
for ro in list(np.linspace(0.8,0.99,70)):
    temp3 = []
    for i in range(15):
        temp3.append(np.mean(POD_simulation(1000000, T0 = 1, q = 3 / 5, Ybar = 10, N = 20, alpha = 0.5, rho = ro)))
    times3.append(np.mean(temp3))
    sd3.append(np.std(temp3)/np.sqrt(15))
    #WE NEED THE MEAN OVERHAD ALSOOOO
    rho.append(ro)    
d3 = {'rho':rho, 'meanDelay':times3, 'sd':sd3}
d3 = pd.DataFrame.from_dict(d3)
d3.to_csv('d3.csv', index = False)

In [ ]:
#STEFANO
rho = []
times4 = []
sd4 = []
for ro in list(np.linspace(0.8,0.99,70)):
    temp4 = []
    for i in range(15):
        temp4.append(np.mean(FAST_simulation(1000000, T0 = 1, q = 3 / 5, Ybar = 10, N = 20, alpha = 0.5, rho = ro)))
    times4.append(np.mean(temp3))
    sd4.append(np.std(temp4)/np.sqrt(15))
    #WE NEED THE MEAN OVERHAD ALSOOOO
    rho.append(ro)    
d4 = {'rho':rho, 'meanDelay':times4, 'sd':sd4}
d4 = pd.DataFrame.from_dict(d4)
d4.to_csv('d4.csv', index = False)

In [ ]:
#stuff to add later
'''Plotting MEAN SYSTEM TIME'''

import matplotlib.pyplot as plt
fig = plt.figure()
#plt.plot(d1['rho'], d1['meanDelay'])#, label = "line 1")
plt.errorbar(d1['rho'],d1['meanOverhead'] , yerr=1.96*d1['sdOver'], label='JSQ-95%CI')
#plt.plot(d2['rho'], d2['meanDelay'])#, label = "line 2")
plt.errorbar(d2['rho'],d2['meanOverhead'] , yerr=1.96*d2['sdOver'], label='JBT-95%CI')
#plt.plot(d3['rho'], d3['meanDelay'])#, label = "line 3")
plt.errorbar(d3['rho'],d3['meanOverhead'] , yerr=1.96*d3['sdOver'], label='POD-95%CI')
plt.legend(loc='lower right', bbox_to_anchor=(0., 0.7, 0.2, 0.2))


plt.savefig('BLA.png')